In [130]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from urllib.parse import urlparse, parse_qs
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
# 篩選出 <a> 並檢查 title 是否以 ".c" 結尾
def has_title_ending_with_c(tag):
    return tag.name == "a" and tag.has_attr("title") and tag["title"].endswith(".c")

def has_href_ending_with_c(tag):
    return tag.name == "a" and tag.has_attr("href") and tag["href"].endswith(".c")

### 參考[chromedriver](https://github.com/dreamshao/chromedriver)下載適合你的chrome版本

In [131]:
#開啟網站
driver_path = r"C:\Users\user\.wdm\drivers\chromedriver\win32\131.0.6778.69\chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)
driver.get("http://www.elearn.ndhu.edu.tw/moodle/login/index.php")
time.sleep(3)

C:\Users\user\AppData\Local\Temp\ipykernel_18484\2940671160.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


### chrome setting(必做)
* 1.進入chrome://settings/content/siteDetails?site=http%3A%2F%2Fwww.elearn.ndhu.edu.tw，"自動下載"改成允許、"不安全內容"改成允許

* 2.進入chrome://settings/downloads，改變下載路徑

In [4]:
#資訊
usser=input("帳戶:")
password=input("密碼:")

In [132]:
#輸入帳密
username_input = driver.find_element(By.NAME, "username")
password_input = driver.find_element(By.NAME, "password")
username_input.send_keys(usser)
password_input.send_keys(password)

In [133]:
#登入
login_button = driver.find_element(By.XPATH, '//input[@type="submit" and @value="登入 "]')
login_button.click()

In [134]:
#點擊 "+" 按鈕
plus_button = driver.find_element(By.XPATH, '//img[@src="http://www.elearn.ndhu.edu.tw/moodle/blocks/myCourses/HTML_TreeMenu-1.2.0/images/plusbottom.gif"]')
plus_button.click()
time.sleep(2)

In [135]:
#點擊課程名稱 "CSIE1040AC-程式設計(一)AC-113上"
course_link = driver.find_element(By.XPATH, '//span[text()="CSIE1040AC-程式設計(一)AC-113上"]')
course_link.click()
time.sleep(2)

In [136]:
#點擊 "作業" 選項
assignment_link = driver.find_element(By.XPATH, '//li[@class="r0"]//a[text()="作業"]')
assignment_link.click()
time.sleep(2) 

In [137]:
# #創檔1~10
# import os
# folder_path=r'C:\Users\user\Documents\cpp\2024 程式設計\2024_final'
# # add new folder in folder_path
# for i in range(1,11):
#     folder_name=f'{i}'
#     folder=os.path.join(folder_path,folder_name)
#     if not os.path.exists(folder):
#         os.makedirs(folder)

In [189]:
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
soup

<html dir="ltr" lang="zh-tw" xml:lang="zh-tw" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="http://www.elearn.ndhu.edu.tw/moodle/theme/standard/styles.php" rel="stylesheet" type="text/css"/>
<link href="http://www.elearn.ndhu.edu.tw/moodle/theme/camo_spring/styles.php" rel="stylesheet" type="text/css"/>
<!--[if IE 7]>
    <link rel="stylesheet" type="text/css" href="http://www.elearn.ndhu.edu.tw/moodle/theme/standard/styles_ie7.css" />
<![endif]-->
<!--[if IE 6]>
    <link rel="stylesheet" type="text/css" href="http://www.elearn.ndhu.edu.tw/moodle/theme/standard/styles_ie6.css" />
<![endif]-->
<meta content="東華e學苑" property="og:title"/>
<meta content="http://www.elearn.ndhu.edu.tw/moodle/pix/icons/ndhu_logo.jpg" property="og:image"/><meta content="國立東華大學數位學習平台" property="og:description"/>
<meta content="moodle, CSIE1080AB-程式設計實驗(一)AB-113上: 作業 " name="keywords"/>
<title>CSIE1080AB-程式設計實驗(一)AB-113上: 作業</title>

In [196]:
#手動填寫題目資訊(事先寫好也行，邏輯還可更自動化)
id=[]#按f12找  觸 檢視n份已繳交作業
for i in range(698463,698473):
    id.append(str(i))
text=[]#份數
text=['73','76','55','67','75','71','62','64','58','58']

In [ ]:
for i in range(10):

    view_submissions = driver.find_element(By.XPATH, f'//a[contains(@href, "submissions.php?id={id[i]}") and text()="檢視 {text[i]} 份已繳交的作業"]')
    view_submissions.click()

    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    result = soup.find_all(has_title_ending_with_c)

    for tag in tqdm(result,desc="Processing"):
        url = tag.get('href', None)
        # 解析 URL
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)

        # 提取 userid 和 file
        userid = query_params.get('userid', [None])[0]
        file = query_params.get('file', [None])[0]

        #點取.c
        c_file_link = driver.find_element(By.XPATH, f'//a[@title="{file}" and contains(@href, "userid={userid}")]')
        c_file_link.click()
        
        #處理新視窗
        time.sleep(2)
        windows = driver.window_handles
        driver.switch_to.window(windows[1])
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        result2 = soup.find_all(has_href_ending_with_c)
        url2=result2[0].get('href', None)
        
        #download
        download_link = driver.find_element(By.XPATH, f'//a[contains(@href, "{url2}")]')
        download_link.click()

        #關掉新視窗 回到原本的
        driver.close()
        driver.switch_to.window(windows[0])
        time.sleep(2)
    #回上頁
    driver.back()

In [ ]:
#檢查
import os
folder_path=r'C:\Users\user\Documents\cpp\2024 程式設計\2024_final'
for folder in os.listdir(folder_path):
    path=os.path.join(folder_path,folder)
    files=os.listdir(path)
    print(f"第{folder}題有: {len(files)} 個人繳交")